In [1]:
import coax
import gym
import haiku as hk
import jax
import jax.numpy as jnp
import optax
from coax.value_losses import mse
from incline_cartpole import InclinedCartpole


# the name of this script
name = 'a2c'

# the cart-pole MDP
env = gym.make('CartPole-v0')
# env = gym.make("rpp_gym:InclinedCartpole-v0")

In [2]:
env.alpha = 0.2

In [3]:
env = coax.wrappers.TrainMonitor(env, name=name, tensorboard_dir=f"./data/tensorboard/{name}")

In [4]:
# env.spec.max_episode_steps = 200
# env.spec.reward_threshold = 195.0

In [5]:
from emlp import T, Scalar
from emlp.groups import SO, S, O, Trivial,Z
from emlp_haiku import EMLPBlock, Sequential, Linear,EMLP, MLP
from emlp.reps import Rep
from emlp.nn import gated,gate_indices,uniform_rep
from math import prod
from representations import PseudoScalar
from mixed_emlp_haiku import MixedEMLP

## Trivial
# group=Trivial(2)
# rep_in = T(0)*prod(env.observation_space.shape)
# rep_out = T(0)*env.action_space.n#prod(env.action_space.shape)

## Reflection
group=Z(2)
rep_in = PseudoScalar()*prod(env.observation_space.shape)
rep_out = T(1)#*env.action_space.n#prod(env.action_space.shape)

# nn_pi = EMLP(rep_in,rep_out,group,ch=100,num_layers=2)
# nn_v = EMLP(rep_in,T(0),group,ch=100,num_layers=2)

# nn_pi = MixedEMLP(rep_in,rep_out(group),group,ch=100,num_layers=2)
# nn_v = MixedEMLP(rep_in,T(0),group,ch=100,num_layers=2)


# def func_pi(S, is_training):
#     return {'logits': nn_pi(S)}


# def func_v(S, is_training):
#     return nn_v(S).reshape(-1)



def func_pi(S, is_training):
    logits = hk.Sequential((
        hk.Linear(16), jax.nn.relu,
        hk.Linear(16), jax.nn.relu,
        hk.Linear(16), jax.nn.relu,
        hk.Linear(env.action_space.n, w_init=jnp.zeros)
    ))
    return {'logits': logits(S)}


def func_v(S, is_training):
    value = hk.Sequential((
        hk.Linear(16), jax.nn.relu,
        hk.Linear(16), jax.nn.relu,
        hk.Linear(16), jax.nn.relu,
        hk.Linear(1, w_init=jnp.zeros), jnp.ravel
    ))
    return value(S)


In [6]:
# these optimizers collect batches of grads before applying updates
optimizer_v = optax.chain(optax.apply_every(k=32), optax.adam(0.002))
optimizer_pi = optax.chain(optax.apply_every(k=32), optax.adam(0.001))


# value function and its derived policy
v = coax.V(func_v, env)
pi = coax.Policy(func_pi, env)

In [7]:
store = v.params

In [8]:
# experience tracer
tracer = coax.reward_tracing.NStep(n=1, gamma=0.9)

# updaters
vanilla_pg = coax.policy_objectives.VanillaPG(pi, optimizer=optimizer_pi)
simple_td = coax.td_learning.SimpleTD(v, loss_function=mse, optimizer=optimizer_v)


# train
for ep in range(1000):
    s = env.reset()
    er = 0
    for t in range(env.spec.max_episode_steps):
        a = pi(s)
        s_next, r, done, info = env.step(a)
        
        if done and (t == env.spec.max_episode_steps - 1):
            r = 1 / (1 - tracer.gamma)
        er+=r
        tracer.add(s, a, r, done)
        while tracer:
            transition_batch = tracer.pop()
            metrics_v, td_error = simple_td.update(transition_batch, return_td_error=True)
            metrics_pi = vanilla_pg.update(transition_batch, td_error)
            env.record_metrics(metrics_v)
            env.record_metrics(metrics_pi)

        if done:
            break

        s = s_next
    
    print("Epoch reward",er)
    # early stopping
    if env.avg_G > env.spec.reward_threshold:
        break


# run env one more time to render
#coax.utils.generate_gif(env, policy=pi, filepath=f"./data/{name}.gif", duration=25)

[a2c|TrainMonitor|INFO] ep: 1,	T: 15,	G: 14,	avg_r: 1,	avg_G: 14,	t: 14,	dt: 370.929ms,	SimpleTD/loss: 0.5,	VanillaPG/loss: 0.693


Epoch reward 14.0


[a2c|TrainMonitor|INFO] ep: 2,	T: 27,	G: 11,	avg_r: 1,	avg_G: 12.5,	t: 11,	dt: 23.628ms,	SimpleTD/loss: 0.5,	VanillaPG/loss: 0.693


Epoch reward 11.0


[a2c|TrainMonitor|INFO] ep: 3,	T: 45,	G: 17,	avg_r: 1,	avg_G: 14,	t: 17,	dt: 24.417ms,	SimpleTD/loss: 0.499,	VanillaPG/loss: 0.692


...

Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 225,	T: 14,018,	G: 200,	avg_r: 1,	avg_G: 194,	t: 200,	dt: 24.688ms,	SimpleTD/loss: 0.000692,	VanillaPG/loss: -0.0022


Epoch reward 209.0


[a2c|TrainMonitor|INFO] ep: 226,	T: 14,219,	G: 200,	avg_r: 1,	avg_G: 195,	t: 200,	dt: 22.953ms,	SimpleTD/loss: 0.000393,	VanillaPG/loss: -0.00264


Epoch reward 209.0
Epoch reward 209.0


In [10]:
coax.utils.dump(pi, "./mlp_pi.lz4")

In [16]:
pi.params['lin']

FlatMapping({
  'linear': FlatMapping({
              'b': DeviceArray([-0.00078749,  0.22243674,  0.24990653,  0.25380114,
                                 0.01685529,  0.02420999,  0.35115007,  0.05400022,
                                 0.21136887,  0.29219517,  0.18837172,  0.22474289,
                                 0.00078776,  0.11020071,  0.35213456,  0.19622795],            dtype=float32),
              'w': DeviceArray([[-0.03512973,  0.05438015, -0.36982128,  0.09777525,
                                 -0.20791632,  0.21831983,  0.05427854,  0.16727945,
                                  0.2172411 ,  0.3122285 , -0.5020807 ,  0.42083606,
                                  0.40071788, -0.6656895 , -0.8764311 , -0.28065702],
                                [-0.27200034, -0.08180515,  0.194785  ,  0.78121173,
                                 -0.86986077, -0.45019484,  0.62767553, -0.7282137 ,
                                 -0.26940435, -0.37256098, -0.19365811, -0.56482714,


In [11]:
loaded_pi = coax.utils.load("./mlp_pi.lz4")

In [13]:
loaded_pi.params

FlatMapping({
  'linear': FlatMapping({
              'b': array([-0.00078749,  0.22243674,  0.24990653,  0.25380114,  0.01685529,
                           0.02420999,  0.35115007,  0.05400022,  0.21136887,  0.29219517,
                           0.18837172,  0.22474289,  0.00078776,  0.11020071,  0.35213456,
                           0.19622795], dtype=float32),
              'w': array([[-0.03512973,  0.05438015, -0.36982128,  0.09777525, -0.20791632,
                            0.21831983,  0.05427854,  0.16727945,  0.2172411 ,  0.3122285 ,
                           -0.5020807 ,  0.42083606,  0.40071788, -0.6656895 , -0.8764311 ,
                           -0.28065702],
                          [-0.27200034, -0.08180515,  0.194785  ,  0.78121173, -0.86986077,
                           -0.45019484,  0.62767553, -0.7282137 , -0.26940435, -0.37256098,
                           -0.19365811, -0.56482714,  1.0171323 , -0.08690273,  0.15433046,
                            0.18701562